# Import des librairies

In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import lxml
from dotenv import load_dotenv
import os


In [7]:
load_dotenv()
blob_keys = os.getenv("AZURE_BLOB_KEYS")

# Fonction de requête  d'un url

In [8]:
def request_get(url):
  """
  Performs a GET request to the given URL and displays the response information.

  Args:
    str: url of the request
  """
  try:
    response = requests.get(url)
    response.raise_for_status()
    return response.text
  except requests.exceptions.RequestException as e:
    return f"Erreur lors de la requête: {e}"
  except ValueError as e:
    return f"Erreur lors du décodage JSON {e}"

# Fonction de récupération de la liste pour un url

In [9]:
def get_list_from_date(year, blob_keys):
    url = f"https://projetocrstorageacc.blob.core.windows.net/invoices-{year}?restype=container&comp=list{blob_keys}"
    content = request_get(url)
    tree = BeautifulSoup(content, features="xml")
    liste = [name.get_text() for name in tree.find_all("Name")]
    return liste
    

In [10]:
print(get_list_from_date("2018", blob_keys))

['FAC_2018_0001-654.png', 'FAC_2018_0002-114.png', 'FAC_2018_0003-025.png', 'FAC_2018_0004-759.png', 'FAC_2018_0005-281.png', 'FAC_2018_0006-250.png', 'FAC_2018_0007-228.png', 'FAC_2018_0008-142.png', 'FAC_2018_0009-754.png', 'FAC_2018_0010-104.png', 'FAC_2018_0011-692.png', 'FAC_2018_0012-758.png', 'FAC_2018_0013-913.png', 'FAC_2018_0014-558.png', 'FAC_2018_0015-089.png', 'FAC_2018_0016-604.png', 'FAC_2018_0017-432.png', 'FAC_2018_0018-032.png', 'FAC_2018_0019-030.png', 'FAC_2018_0020-095.png', 'FAC_2018_0021-223.png', 'FAC_2018_0022-238.png', 'FAC_2018_0023-517.png', 'FAC_2018_0024-616.png', 'FAC_2018_0025-027.png', 'FAC_2018_0026-574.png', 'FAC_2018_0027-203.png', 'FAC_2018_0028-733.png', 'FAC_2018_0029-665.png', 'FAC_2018_0030-718.png', 'FAC_2018_0031-558.png', 'FAC_2018_0032-429.png', 'FAC_2018_0033-225.png', 'FAC_2018_0034-459.png', 'FAC_2018_0035-603.png', 'FAC_2018_0036-284.png', 'FAC_2018_0037-828.png', 'FAC_2018_0038-890.png', 'FAC_2018_0039-006.png', 'FAC_2018_0040-777.png',

In [11]:
all_files = []
for year in range(2018, 2026):
    year_files = get_list_from_date(year, blob_keys)
    all_files.extend(year_files)
print(all_files)

['FAC_2018_0001-654.png', 'FAC_2018_0002-114.png', 'FAC_2018_0003-025.png', 'FAC_2018_0004-759.png', 'FAC_2018_0005-281.png', 'FAC_2018_0006-250.png', 'FAC_2018_0007-228.png', 'FAC_2018_0008-142.png', 'FAC_2018_0009-754.png', 'FAC_2018_0010-104.png', 'FAC_2018_0011-692.png', 'FAC_2018_0012-758.png', 'FAC_2018_0013-913.png', 'FAC_2018_0014-558.png', 'FAC_2018_0015-089.png', 'FAC_2018_0016-604.png', 'FAC_2018_0017-432.png', 'FAC_2018_0018-032.png', 'FAC_2018_0019-030.png', 'FAC_2018_0020-095.png', 'FAC_2018_0021-223.png', 'FAC_2018_0022-238.png', 'FAC_2018_0023-517.png', 'FAC_2018_0024-616.png', 'FAC_2018_0025-027.png', 'FAC_2018_0026-574.png', 'FAC_2018_0027-203.png', 'FAC_2018_0028-733.png', 'FAC_2018_0029-665.png', 'FAC_2018_0030-718.png', 'FAC_2018_0031-558.png', 'FAC_2018_0032-429.png', 'FAC_2018_0033-225.png', 'FAC_2018_0034-459.png', 'FAC_2018_0035-603.png', 'FAC_2018_0036-284.png', 'FAC_2018_0037-828.png', 'FAC_2018_0038-890.png', 'FAC_2018_0039-006.png', 'FAC_2018_0040-777.png',

In [12]:
len(all_files)

6122

# Téléchargement des fichiers

In [27]:
def download_file_requests_os(url, filename):
    """
    Télécharge un fichier à partir d'une URL en utilisant requests et os, et crée les dossiers nécessaires.

    Args:
        url: L'URL du fichier à télécharger.
        filename: Le nom du fichier sous lequel enregistrer le fichier.
    """
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        directory = os.path.dirname(filename)
        if directory and not os.path.exists(directory):
            os.makedirs(directory, exist_ok=True)
        with open(filename, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors du téléchargement du fichier (méthode 3) : {e}")
    except IOError as e:
        print(f"Erreur lors de l'enregistrement du fichier (méthode 3): {e}")
    except Exception as e:
      print(f"Erreur (méthode 3): {e}")

In [28]:
count = 0
for file in all_files:
    year = file.split("_")[1]
    folder_path = f"../data/files/{year}"
    os.makedirs(folder_path, exist_ok=True)
    url = f"https://projetocrstorageacc.blob.core.windows.net/invoices-{year}/{file}?{blob_keys}"
    download_file_requests_os(url, f"{folder_path}/{file}")
    count += 1
    if count % 100 == 0:
        print(count)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
